## Forest Cover Classification

In this project, you will use deep learning to predict forest cover type (the most common kind of tree cover) based only on cartographic variables. The actual forest cover type for a given 30 x 30 meter cell was determined from US Forest Service (USFS) Region 2 Resource Information System data. The covertypes are the following:

- Spruce/Fir
- Lodgepole Pine
- Ponderosa Pine
- Cottonwood/Willow
- Aspen
- Douglas-fir
- Krummholz


Independent variables were then derived from data obtained from the US Geological Survey and USFS. The data is raw and has not been scaled or preprocessed for you. It contains binary columns of data for qualitative independent variables such as wilderness areas and soil type.

This study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado. These areas represent forests with minimal human-caused disturbances, so existing forest cover types are mainly a result of ecological processes rather than forest management practices.

Project Objectives:

Develop one or more classifiers for this multi-class classification problem.
Use TensorFlow with Keras to build your classifier(s).
Use your knowledge of hyperparameter tuning to improve the performance of your model(s).
Test and analyze performance.
Create clean and modular code.

In [63]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import numpy as np

# Load data
data = pd.read_csv('cover_data.csv')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           581012 non-null  int64
 1   Aspect                              581012 non-null  int64
 2   Slope                               581012 non-null  int64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64
 6   Hillshade_9am                       581012 non-null  int64
 7   Hillshade_Noon                      581012 non-null  int64
 8   Hillshade_3pm                       581012 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64
 10  Wilderness_Area1                    581012 non-null  int64
 11  Wilderness_Area2                    581012 non-null 

In [64]:
# Print distribution of class column (labels)
print('Classes and number of values in the dataset', Counter(data['class']))

Classes and number of values in the dataset Counter({2: 283301, 1: 211840, 3: 35754, 7: 20510, 6: 17367, 5: 9493, 4: 2747})


In [65]:
# Split data into label and feature sets
y = data['class']
x = data[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 
          'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
          'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3',
          'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 
          'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 
          'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 
          'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 
          'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']]      

In [66]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42)

In [67]:
# Standardize numeric features
column_transformer = ColumnTransformer([('numeric', StandardScaler(), ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 
          'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
          'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3',
          'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 
          'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 
          'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 
          'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 
          'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40'])])

X_train = column_transformer.fit_transform(X_train)
X_test = column_transformer.transform(X_test)

In [68]:
# Encode labels
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train.astype(str))
y_test = label_encoder.transform(y_test.astype(str))

In [69]:
# Transform encoded labels into binary vectors
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [70]:
# Build neural network
model = Sequential()
model.add(InputLayer(input_shape = (X_train.shape[1], )))

# Hidden Layers
model.add(Dense(64, activation='relu'))  
model.add(BatchNormalization())
model.add(Dropout(0.5))  
model.add(Dense(32, activation='relu'))  
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(7, activation = 'softmax'))

In [71]:
# Compile model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [72]:
# Callback
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10)

# Train and evaluate model
model.fit(X_train, y_train, validation_split = 0.2, epochs = 100, batch_size = 32, callbacks = [early_stopping])
loss, accuracy = model.evaluate(X_train, y_train, verbose = 0)
print("Loss: ", loss, "Accuracy: ", accuracy)

Epoch 1/100
11621/11621 [==============================] - 6s 526us/step - loss: 0.7857 - accuracy: 0.6809 - val_loss: 0.6085 - val_accuracy: 0.7421
Epoch 2/100
11621/11621 [==============================] - 6s 523us/step - loss: 0.6821 - accuracy: 0.7169 - val_loss: 0.5873 - val_accuracy: 0.7528
Epoch 3/100
11621/11621 [==============================] - 6s 520us/step - loss: 0.6639 - accuracy: 0.7230 - val_loss: 0.5760 - val_accuracy: 0.7554
Epoch 4/100
11621/11621 [==============================] - 6s 519us/step - loss: 0.6561 - accuracy: 0.7253 - val_loss: 0.5687 - val_accuracy: 0.7602
Epoch 5/100
11621/11621 [==============================] - 6s 519us/step - loss: 0.6497 - accuracy: 0.7290 - val_loss: 0.5635 - val_accuracy: 0.7670
Epoch 6/100
11621/11621 [==============================] - 6s 520us/step - loss: 0.6457 - accuracy: 0.7300 - val_loss: 0.5608 - val_accuracy: 0.7655
Epoch 7/100
11621/11621 [==============================] - 6s 522us/step - loss: 0.6454 - accuracy: 0.7309

In [73]:
# Classification report
y_estimate = model.predict(X_test, verbose = 0)
y_estimate = np.argmax(y_estimate, axis = 1)
y_true = np.argmax(y_test, axis = 1)
print(classification_report(y_true, y_estimate))

              precision    recall  f1-score   support

           0       0.75      0.70      0.72     42557
           1       0.73      0.89      0.80     56500
           2       0.68      0.70      0.69      7121
           3       0.00      0.00      0.00       526
           4       0.00      0.00      0.00      1995
           5       0.87      0.05      0.09      3489
           6       0.99      0.04      0.07      4015

    accuracy                           0.73    116203
   macro avg       0.57      0.34      0.34    116203
weighted avg       0.73      0.73      0.70    116203



/opt/homebrew/Cellar/jupyterlab/4.0.7_1/libexec/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/Cellar/jupyterlab/4.0.7_1/libexec/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/Cellar/jupyterlab/4.0.7_1/libexec/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf